# Legal Robots: XML-Scraping

Für die Aufgabe benötigt **A.A.** zunächst einmal Daten. 
Zwar finden sich in der Bibliothek Unmengen davon; 
er rechnet aber nicht damit, einen der seltenen Scanroboter dauerhaft für sein Projekt einspannen zu können. 
Außerdem sind die Informationen in Büchern ja ganz unterschiedlich strukturiert und daher schwer auszuwerten.

Da erinnert sich **A.A.** an die Webseite [Rechtsprechung im Internet](https://www.rechtsprechung-im-internet.de), 
auf der sich zahlreiche Entscheidungen der obersten Gerichte abrufen lassen. 
Über den Menüpunkt *Hinweise* erhält er ein **aktuelles Inhaltsverzeichnis** der Entscheidungen im XML-Format, 
das er sich herunterlädt und ansieht.

In [ ]:
<?xml version='1.0' encoding='UTF-8'?>
<!DOCTYPE items [
 <!ELEMENT items (item)*>
 <!ELEMENT item (gericht?,entsch-datum?,aktenzeichen?,link,modified)>
 <!ELEMENT gericht (#PCDATA)>
 <!ELEMENT entsch-datum (#PCDATA)>
 <!ELEMENT aktenzeichen (#PCDATA)>
 <!ELEMENT link (#PCDATA)>
 <!ELEMENT modified (#PCDATA)>
]>
<!-- generator:jps20-V30_02_00 20.02.2017 10:38, lastBuildDate:24.03.2017 23:47:24 -->
<items>
<item>
 <gericht>BGH 9. Zivilsenat</gericht>
 <entsch-datum>20100114</entsch-datum>
 <aktenzeichen>IX ZB 72/08</aktenzeichen>
 <link>http://www.rechtsprechung-im-internet.de/jportal/docs/bsjrs/JURE100055033.zip</link>
 <modified>2016-01-25T22:46:20Z</modified>
</item>
<item>
 <gericht>BGH 9. Zivilsenat</gericht>
 <entsch-datum>20100114</entsch-datum>
 <aktenzeichen>IX ZB 76/09</aktenzeichen>
 <link>http://www.rechtsprechung-im-internet.de/jportal/docs/bsjrs/JURE100055035.zip</link>
 <modified>2016-01-25T22:46:20Z</modified>
</item>
<item>
 <gericht>BGH 2. Strafsenat</gericht>
 <entsch-datum>20100113</entsch-datum>
 <aktenzeichen>2 ARs 569/09</aktenzeichen>
 <link>http://www.rechtsprechung-im-internet.de/jportal/docs/bsjrs/JURE100055051.zip</link>
 <modified>2016-01-25T22:46:20Z</modified>
</item>
<item>
 <gericht>BGH 2. Strafsenat</gericht>
 <entsch-datum>20100113</entsch-datum>
 <aktenzeichen>2 StR 428/09</aktenzeichen>
 <link>http://www.rechtsprechung-im-internet.de/jportal/docs/bsjrs/JURE100055072.zip</link>
 <modified>2016-01-25T22:46:20Z</modified>
</item>
<!-- ... -->
</items>

#### Aufgabe 1: Code verstehen

a) Schau dir die abgebildeten 42 Zeilen der Datei https://www.rechtsprechung-im-internet.de/rii-toc.xml an. 
Welche Regelmäßigkeiten stellst du fest?
    
b) Was soll wohl der Teil, der mit `<!DOCTYPE` beginnt?

c) Besuche einen der Links aus dem Inhaltsverzeichnis und sieh dir an, was du dort erhältst. 
Was müsste man tun, um diese Daten in auswertbarer Form für sich selbst dauerhaft zu speichern?
________

Mit der Hilfe eines `Scraping-Roboters` möchte **A.A.** auf der Grundlage dieser Entscheidungen eine eigene Datenbasis erstellen. Die folgende Klasse zeigt den Aufbau eines **RII-Scraping-Roboters**:

In [ ]:
import urllib.request, zipfile, io, os
from bs4 import BeautifulSoup


class RIIScrapingRobot:
    
    def __init__(self, name: str):
        self.name = name
        self.skill = "Download von RII-Entscheidungen"
        
        tocfile = open("rii-toc.xml","r")
        toc = BeautifulSoup(tocfile, 'xml')
        self.decisions = toc.find_all('item')
    
    def show_decision_count(self):
        # TODO: Implement (Aufgabe 2a)
        pass
    
    def find_latest_decision(self):
        latest = self.decisions[0]
        for decision in self.decisions:
            if decision.find('entsch-datum').get_text() > latest.find('entsch-datum').get_text():
                latest = decision
        
        return latest
        
    def show_latest_decision(self):
        latest = self.find_latest_decision()
    
        latest_gericht = latest.gericht.get_text()
        latest_az = latest.aktenzeichen.get_text()
        latest_datum = latest.find('entsch-datum').get_text()
        
        print("Die neueste Entscheidung vom {} kommt von {} und hat das Aktenzeichen {}.".format(
            latest_datum, latest_gericht, latest_az))
           
    def show_latest_text(self):
        latest = self.find_latest_decision()
        link = latest.link.get_text()
        
        response = urllib.request.urlopen(link)
        
        ziphandler = zipfile.ZipFile(io.BytesIO(response.read()))
        filename = ziphandler.namelist()[0]
        xmltext = ziphandler.read(filename)
        
        textsoup = BeautifulSoup(xmltext, "xml")
        
        print(textsoup.get_text())
    
    
    def save_all_decisions(self):
        # TODO: Implement (Aufgabe 2e)
        pass

In [ ]:
# TODO: Roboter "Scraper" erzeugen (Aufgabe 2b)

#### Aufgabe 2: Code weiterentwickeln

a) Der Roboter soll bei Aufruf der Methode `show_decision_count()` ausgeben, 
wie viele Entscheidungen im Inhaltsverzeichnis gefunden wurden. Implentiere diese Methode.

b) Erstelle einen Scraping Roboter mit dem Namen `Scraper` und lasse dir von ihm die wichtigsten Informationen über die neueste Entscheidung aus dem Inhaltsverzeichnis ausgeben. 
Lässt sich wirklich eindeutig *eine* neueste Entscheidung ermitteln?
Überlege dir eine bessere Lösung für dieses Problem und implementiere diese.

c) Betrachte nochmal die Ausgabe von `Scraper` aus Teilaufgabe b). 
Was stört dich noch an dieser Ausgabe? 
Wie lässt sich das Problem beheben?

d) Unser Roboter verwendet eine Inhaltsverzeichnisdatei, die sich auf der Festplatte befindet. 
Erweitere den Code so, dass immer auf die jeweils aktuelle Datei bei *Rechtsprechung im Internet* zugegriffen wird.
> **Tipp:** Eine Inspiration sollte dir die Methode `show_latest_text()` bieten.

e) Der RII-Scraping-Roboter soll bei Aufruf der Methode `save_all_decisions()` die XML-Dateien aller Entscheidungen abrufen und in einen Ordner `rii-files/` ablegen. 
Vervollständige diese Methode und lasse `Scraper` sämtliche Dateien abrufen.
> **Tipp:** Während du den Code entwickelst und testest, 
kannst du mit `for element in liste[:5]` anstelle von `for element in liste` nur die ersten fünf Elemente einer Liste durchlaufen.
Damit kannst du den Code mit einer kleinen Testmenge prüfen und erst den korrekten Code auf alle Elemente anwenden.

> **Tipp:** Die Methode [`extractall(path)`](https://docs.python.org/2/library/zipfile.html) des zipfile-Objekts entpackt alle Dateien des Archivs in den als `path` übergebenen Ordner.